In [9]:
%matplotlib inline
import pandas as pd
import numpy as np
import artpop
import astropy.units as u
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.visualization import make_lupton_rgb
from random import gauss

In [17]:
artpop.filters.phot_system_lookup()

{'UKIDSS_Z': 'UKIDSS',
 'UKIDSS_Y': 'UKIDSS',
 'UKIDSS_J': 'UKIDSS',
 'UKIDSS_H': 'UKIDSS',
 'UKIDSS_K': 'UKIDSS',
 'R062': 'WFIRST',
 'Z087': 'WFIRST',
 'Y106': 'WFIRST',
 'J129': 'WFIRST',
 'W146': 'WFIRST',
 'H158': 'WFIRST',
 'F184': 'WFIRST',
 'LSST_u': 'LSST',
 'LSST_g': 'LSST',
 'LSST_r': 'LSST',
 'LSST_i': 'LSST',
 'LSST_z': 'LSST',
 'LSST_y': 'LSST',
 'DECam_u': 'DECam',
 'DECam_g': 'DECam',
 'DECam_r': 'DECam',
 'DECam_i': 'DECam',
 'DECam_z': 'DECam',
 'DECam_Y': 'DECam',
 'SDSS_u': 'SDSSugriz',
 'SDSS_g': 'SDSSugriz',
 'SDSS_r': 'SDSSugriz',
 'SDSS_i': 'SDSSugriz',
 'SDSS_z': 'SDSSugriz',
 'WFC3_UVIS_F200LP': 'HST_WFC3',
 'WFC3_UVIS_F218W': 'HST_WFC3',
 'WFC3_UVIS_F225W': 'HST_WFC3',
 'WFC3_UVIS_F275W': 'HST_WFC3',
 'WFC3_UVIS_F280N': 'HST_WFC3',
 'WFC3_UVIS_F300X': 'HST_WFC3',
 'WFC3_UVIS_F336W': 'HST_WFC3',
 'WFC3_UVIS_F343N': 'HST_WFC3',
 'WFC3_UVIS_F350LP': 'HST_WFC3',
 'WFC3_UVIS_F373N': 'HST_WFC3',
 'WFC3_UVIS_F390M': 'HST_WFC3',
 'WFC3_UVIS_F390W': 'HST_WFC3',
 'WFC3

In [136]:
ssp = artpop.MISTSSP(
    log_age = np.log10(0.9e9),       # log of age in years
    feh = -1.577,           # metallicity [Fe/H]
    distance = 22*u.Mpc,
    phot_system = 'HST_ACSWF', # photometric system(s)
    total_mass = 2.4e6,
    imf = 'kroupa'
#     num_stars = 5e5,      # number of stars
#     random_state = rng,   # random state for reproducibility
)

In [124]:
def check_MZR(FeH):
    #for luminosity-weighted metallicity based on total stellar mass; not worrying about enrichment
    if FeH <=  -0.8:  #Kirby  et  al. 2013 https://ui.adsabs.harvard.edu/abs/2013ApJ...779..102K/abstract
        logmass = (FeH + 1.69)/0.3  + 6
        
        return gauss(logmass, 0.2)
        
    if FeH > -0.8:  #adapted from Gallazzi et al. 2005 https://ui.adsabs.harvard.edu/abs/2005MNRAS.362...41G/abstract
        logmass_ = [8.91, 9.11, 9.31, 9.51, 9.72, 9.91, 10.11, 10.31, 10.51, 10.72, 10.91, 11.11, 11.31, 11.51, 11.72, 11.91]
        FeH_ = [-0.60, -0.61, -0.65, -0.61, -0.52, -0.41, -0.23, -0.11, -0.01, 0.04, 0.07, 0.1, 0.12, 0.13, 0.14, 0.15]
        MZR_fit = np.polyfit(np.array(FeH_)[logmass_ >= 9.3], np.array(logmass_)[logmass_ >= 9.3], 6)
        
        logmass = (MZR_fit[0]*FeH**6 + MZR_fit[1]*FeH**5 + MZR_fit[2]*FeH**4 + 
                   MZR_fit[3]*FeH**3 + MZR_fit[4]*FeH**2 + MZR_fit[5]*FeH**1 + MZR_fit[6])
        
        return gauss(logmass, 0.3)   

In [ ]:
def get_lum(ssp, filt, system):
    """
    Get integrated luminosity of the galaxy.
    
    PARAMETERS:
        
    """

In [ ]:
def get_lwFeH(ssp1, ssp2, ssp3, filt):
    """
    returns total stellar mass inferred from the luminoity-weighted [Fe/H]
    """
    total_lum = 
    lwfeh = 10**ssp1.total_mag(filt)*ssp1.feh + *ssp2.feh + *ssp3.feh/()
    
    return check_MZR(lwfeh)

In [54]:
def perturb_mags(model_filts, input_colors, input_mags, input_color_err, input_mag_err):
    """
    Make model CMD more realistic by 1. matching the limiting magnitude and 2. perturbing ArtPop stellar magnitudes by the errors on the input CMD.
    
    PARAMETERS:
        model_filts (str, arr-like): list of filters used in CMD -- [bluer filter, redder filter], e.g. ["ACS_WFC_F606W", "ACS_WFC_F814W"]
        input_colors (arr-like): x-axis of data-based CMD, e.g. F606W-F814W
        input_mags (arr-like): y-axis of data-based CMD, e.g. F814W
        input_color_errs (arr-like): error on x-axis of data-based CMD
        input_mag_errs (arr-like): error y-axis of data-based CMD
    
    
    RETURNS: 
        CMD x-axis (perturbed model colors, array), CMD y-axis (perturbed model magnitudes, array) 
    """
    
    pop = np.array(ssp.star_mags(model_filts[1])[ssp.star_mags(model_filts[1]) <= (max(input_mags) + max(input_mag_err))])
    pop_color = np.array((ssp.star_mags(model_filts[0]) - ssp.star_mags(model_filts[1]))[ssp.star_mags(model_filts[1]) <= (max(input_mags) + max(input_mag_err))])

    pop_err = []
    pop_color_err = []
    new_pop = []
    new_popcolor = []

    for i in range(len(pop)):
        pop_err_mod = input_mag_err[np.argmin(abs(input_mags - pop[i]))]
        pop_err.append(pop_err_mod)
        new_pop.append(pop[i] + gauss(0, pop_err_mod**2))

        pop_color_err_mod = input_mag_err[np.argmin(abs(input_mags - pop[i]))]
        pop_color_err.append(pop_color_err_mod)
        new_popcolor.append(pop_color[i] + gauss(0, pop_color_err_mod**2))
        
    return new_popcolor, new_pop

In [ ]:
def plot_results(img_filts, reff, n, theta, ellip, distance, pixel_scale, CMD_filts = img_filts[-2:], zpts = None, psfs = None, xy_dim = 1001, Q = 0.5, stretch = 0.06, save = True)
    stars = csp.abs_mag_table

    xy = artpop.sersic_xy(
        num_stars = len(stars),     
        r_eff = reff*u.kpc,         # effective radius (kpc)
        n = n,             # Sersic index
        theta = theta*u.deg,          # position angle (deg)
        ellip = ellip,         # ellipticity
        distance = distance*u.Mpc,       # distance to system (Mpc)
        xy_dim = xy_dim, # xy dimensions of image
        pixel_scale = pixel_scale
    )
    
    distance_mod = 5*np.log10(distance*u.Mpc.to(u.pc)/10)
    
    mags_ = {str(img_filts[0]): stars[str(img_filts[0])] + distance_mod, str(img_filts[1]): stars[str(img_filts[1])] + distance_mod, 
         str(img_filts[2]): stars[str(img_filts[2])] + distance_mod} #might need to convert names/between abs mag and app mag; will need to see as we work on it
    
    ssp = artpop.source.Source(xy, mags = mags_, xy_dim = 1001, pixel_scale = 0.05)
    
    img = {}
    filt = img_filts
    zpt_ = zpts
    psf_ = psfs
    for i in range(len(filt)):
        img[filt[i]] = imager.observe(ssp, bandpass = img_filt[i], psf = psf_[i], zpt = zpt_[i]).image
    
    if (len(img_filts) > 3) or (len(img_filts) < 2):
        raise Exception("Must have two or three filters to generate mock image.")
    
    if len(img_filts) == 2:
        b = img[img_filts[0]]
        g = (img[img_filts[0]] + img[img_filts[1]])/2
        r = img[img_filts[1]]
        
    else:
        b = img[img_filts[0]]
        g = img[img_filts[1]]
        r = img[img_filts[2]]

    rgb = make_lupton_rgb(r, g, b, Q=Q, stretch=stretch)
    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (30, 10))
    
    ax1.scatter(dict_data, dict_data, color = 'k', s = 10)
    ax1.set_xlabel(CMD_filts[0] + '-' + CMD_filts[1])
    ax1.set_ylabel(CMD_filts[1])
    ax1.set_title('input CMD')
    ax1.invert_yaxis()
    
    ax2.scatter(mags_[CMD_filts[0]] - mags_[CMD_filts[1]], mags_[CMD_filts[1]], color = 'k', s = 10)
    ax2.set_xlabel(CMD_filts[0] + '-' + CMD_filts[1])
    ax2.set_ylabel(CMD_filts[1])
    ax2.set_title('synthetic CMD')
    ax2.invert_yaxis()
    
    ax3.imshow(rgb, origin = 'lower')
    ax3.axis('off')
    
    if save == True:
        fig.savefig('FitAP_out.pdf', bbox_inches = 'tight', dpi = 200)